In [ ]:
!pip install emoji
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from google.colab import drive
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 358.9/358.9 kB 6.7 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# loading the dataset
drive.mount('/content/drive')
dataset = '/content/drive/MyDrive/Colab/train_hindi_mod.csv'
df1 = pd.read_csv(dataset)

en_hn_dict = '/content/drive/MyDrive/Colab/hindi_english_mod2.csv'
df_trans = pd.read_csv(en_hn_dict)

bad_words = '/content/drive/MyDrive/Colab/bad_words.csv'
with open(bad_words, 'r') as file:
    offensive_words = file.read().split(',')

# converting dataframe to dictionary
post_dict = df1.to_dict('records')
trans_dict = df_trans.to_dict('records')

Mounted at /content/drive


In [ ]:
# creating the dictionary for hindi to english
dictionary = {}

for row in trans_dict:
    if row['hindi'] not in dictionary:
        dictionary[row['hindi']] = [str(row['english'])]
    else:
        dictionary[row['hindi']].append(str(row['english']))
print("Dictionary size: ", len(dictionary))

Dictionary size:  32680


In [ ]:
# preprocessing the data
def preprocess_sentence(sentence):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Tokenize the sentence
    tokens = word_tokenize(sentence.lower())

    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Stem each word
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatize each word
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

    # Join the tokens back into a sentence
    cleaned_sentence = ' '.join(lemmatized_tokens)

    return cleaned_sentence

In [ ]:
def filtertext(sentnc):
    sentnc = re.sub(r'\bhttp\S+', '', sentnc) # Remove links
    # text = re.sub(r'\B#\w+', '', text) # Remove hashtags
    sentnc = re.sub(r'\B@\w+', '', sentnc) # Remove mentions
    sentnc = emoji.demojize(sentnc) # Replace emojis with text representation
    sentnc = re.sub(r':[a-z_]+:', '', sentnc) # Remove emoji codes
    words = re.split(r'[-, :0-9\n?\'\"]+', sentnc) # splitting based on delimiter
    words = list(filter(lambda word: word != "", words))
    sentnc = ' '.join(words)
    return sentnc

In [ ]:
#translating the sentence
def translate(sentence):
    global dictionary
    words = sentence.split()
    new_sentence = ''
    for word in words:
        if word not in dictionary:
            new_sentence += word + ' '
        else:
            # for i in range(min(4, len(dictionary[word]))):
            for meaning in dictionary[word]:
                new_sentence += meaning + ' '
    return new_sentence.rstrip()


# translating and expanding the query
def transexpand(sentence):
    global dictionary
    words = sentence.split()
    new_sentence = ''
    for word in words:
        if word not in dictionary:
            new_sentence += word + ' '
        else:
            for i in range(min(4, len(dictionary[word]))):
                new_sentence += dictionary[word][i] + ' '
    return new_sentence.rstrip()

In [ ]:
# splitting the data in test and train
X_train, X_test, y_train, y_test = train_test_split(df1['Post'], df1['Labels Set'], test_size=0.2, random_state= 9)

# filtering, translating and preprocessing the training data
X_train = X_train.apply(filtertext)
X_train = X_train.apply(translate)
X_train = X_train.apply(preprocess_sentence)


# initialising the lists and dicts
class_docs_totals = {} # to store how many docs a word appeared in a class
class_word_counts = {} # to sotre how many times a word appeared in a class
class_word_totals = {} # total no. of words in a class
class_doc_counts = {} # no. of doc in a class containing the word
class_priors = {} # probability of a class
vocab = set() # vocabulary generated from the training data

# print the training data metadata
count_1 = list(y_train).count(1)
count_0 = len(y_train) - count_1
print("Training data size: ", len(y_train))
print("Label Imbalance: ", count_1 / count_0)

Training data size:  4582
Label Imbalance:  0.297281993204983


In [ ]:
# the method Used:  score = Summation of prob
# Define a function to predict the class of a new text sample
def predict2(text):
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    text = filtertext(text)
    words = text.split()
    for word in words:
          if word in offensive_words:
              return 1
    text = transexpand(text)
    text = preprocess_sentence(text)
    probs = {}
    for c in np.unique(y_train):
        log_prob = 0
        # log_prob = 1;
        for word in words:
          count = 1  # Laplace smoothing
          if word in vocab:
              count += class_doc_counts[c][word]
          log_prob += np.log(count / (class_docs_totals[c] + len(vocab)))
        probs[c] = log_prob
    # return max(probs, key=probs.get)
    if(probs[1] > 0.85 * probs[0]):
        return 1
    return 0

In [ ]:
# the method Used:  score = Summation of prob with prior
# Define a function to predict the class of a new text sample
def predict3(text):
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    # text = preprocess_sentence(text)
    text = filtertext(text)
    words = text.split()
    for word in words:
          if word in offensive_words:
              return 1
    text = transexpand(text)
    text = preprocess_sentence(text)
    probs = {}
    for c in np.unique(y_train):
        log_prob = np.log(class_priors[c])
        # log_prob = 1;
        for word in words:
          count = 1  # Laplace smoothing
          if word in vocab:
              count += class_word_counts[c][word]
          log_prob += np.log(count / (class_word_totals[c] + len(vocab)))
        probs[c] = log_prob
    # return max(probs, key=probs.get)
    if(probs[1] > 0.85 * probs[0]):
        return 1
    return 0

In [ ]:
# the method Used:  score = Summation of (1 + log(tf))
# Define a function to predict the class of a new text sample
def predict(text):
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    text = filtertext(text)
    words = text.split()
    for word in words:
          if word in offensive_words:
              return 1
    text = transexpand(text)
    text = preprocess_sentence(text)
    probs = {}
    for c in np.unique(y_train):
        score = 0
        for word in words:
          if word in vocab:
            if class_word_counts[c][word] != 0:
              score += (1 + np.log(class_word_counts[c][word]))
            else:
              score += 0
        probs[c] = score
    # return max(probs, key=probs.get)
    if(probs[1] > 0.85 * probs[0]):
        return 1
    return 0

In [ ]:
def basemodel():
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    # evaluting the vocabulary
    for sentence in X_train:
        words = sentence.split()
        for word in words:
            vocab.add(word)

    print("Vocab size: ", len(vocab))

    # Create a dictionary to store the count of each word in each class
    # array of index = word for all labels
    # count how many time a word occured in each labels
    y_train_unique = np.unique(y_train)
    for c in y_train_unique:
        class_doc_counts[c] = {}
        class_word_counts[c] = {}
        for word in vocab:
            class_doc_counts[c][word] = 0
            class_word_counts[c][word] = 0

    # Count the number of occurrences of each word in each class
    for sent, c in zip(X_train, y_train):
        words = sent.split()
        for word in words:
            class_word_counts[c][word] += 1
        for word in np.unique(words):
            class_doc_counts[c][word] += 1

    for c in np.unique(y_train):
        class_priors[c] = (len(y_train[y_train == c]) + 1) / (len(y_train) + len(np.unique(y_train)))

    for c in np.unique(y_train):
        class_docs_totals[c] = sum(class_doc_counts[c].values())

    # Compute the total count of words in each class
    for c in y_train_unique:
        class_word_totals[c] = sum(class_word_counts[c].values())

In [ ]:
basemodel()
correct = 0
tp = 0
fp = 0
tn = 0
fn = 0
for sen, off in zip(X_test, y_test):
    pred = predict(sen)
    if pred == 1:
      if(off == 1):
        tp += 1
        correct += 1
      else:
        fp += 1
    else:
      if(off == 0):
        correct += 1
        tn += 1
      else:
        fn += 1
accuracy = correct / len(y_test)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Accuracy:", accuracy)
print("Percision:", precision)
print("Recall:", recall)
f1_score = (2 * precision * recall) / (precision + recall)
print("F1 score:", f1_score)

Vocab size:  21259
Accuracy: 0.8219895287958116
Percision: 0.7317073170731707
Recall: 0.42857142857142855
F1 score: 0.5405405405405405


In [ ]:
print(predict('''#‡§ú‡•Ä‡§µ‡§®‡§∏‡§Ç‡§µ‡§æ‡§¶: ‡§π‡§Æ ‡§∏‡§Ç‡§ò‡§∞‡•ç‡§∑ ‡§ï‡§∞‡§®‡§æ ‡§ö‡§æ‡§π‡§§‡•á ‡§π‡•à‡§Ç, ‡§ï‡•ç‡§Ø‡•ã‡§Ç‡§ï‡§ø ‡§∏‡§æ‡§Æ‡§®‡•á ‡§∏‡§Ç‡§ò‡§∞‡•ç‡§∑ ‡§ï‡•Ä ‡§Æ‡§ø‡§∏‡§æ‡§≤ ‡§∞‡§ñ‡§®‡§æ ‡§ö‡§æ‡§π‡§§‡•á ‡§π‡•à‡§Ç. ‡§®‡§è ‡§∂‡§π‡§∞ ‡§Æ‡•á‡§Ç ‡§á‡§§‡§®‡§æ ‡§ï‡•Å‡§õ ‡§®‡§Ø‡§æ ‡§π‡•ã‡§ó‡§æ ‡§ï‡§ø ‡§π‡§Æ‡•á‡§Ç ‡§Ö‡§™‡§®‡•Ä ‡§ú‡§°‡§º‡•á‡§Ç ‡§ú‡§Æ‡§æ‡§®‡•á ‡§Æ‡•á‡§Ç ‡§¨‡§∞‡§∏‡•ã‡§Ç ‡§¨‡•Ä‡§§ ‡§ú‡§æ‡§è‡§Ç‡§ó‡•á. ‡§π‡§Æ ‡§ï‡§∞‡•ç‡§ú ‡§Æ‡•á‡§Ç ‡§ú‡§∞‡•Ç‡§∞ ‡§π‡•à‡§Ç, ‡§≤‡•á‡§ï‡§ø‡§® ‡§π‡§Æ ‡§π‡§æ‡§∞‡§®‡§æ ‡§®‡§π‡•Ä‡§Ç ‡§ö‡§æ‡§π‡§§‡•á. ‡§µ‡§π ‡§≠‡•Ä ‡§¨‡§ø‡§®‡§æ ‡§≤‡§°‡§º‡•á.
#JeevanSamvad
@DayashankarMi
https://t.co/nd7troF0JF https://t.co/CmAzq2Mj8S'''))
print(predict('''‡§Æ‡•ã‡§¶‡•Ä‡§ú‡•Ä ‡§î‡§∞ ‡§ú‡§¨ ‡§∏‡§æ‡§∞‡§æ ‡§¶‡•á‡§∂ ‡§∏‡•á‡§®‡§æ ‡§ï‡•á ‡§∏‡§æ‡§• ‡§ñ‡§°‡•Ä ‡§π‡•à,‡§™‡§∞ ‡§¶‡•ã ‡§∏‡§Ø‡§æ‡§®‡•á ‡§µ‡§ø‡§¶‡•á‡§∂ ‡§Æ‡•á ‡§™‡§°‡•á ‡§π‡•à?ü§î ‡§á‡§∏‡§≤‡§ø‡§è ‡§¨‡•ã‡§≤‡§§‡•á ‡§π‡•à‡§Ç ‡§µ‡§ø‡§¶‡•á‡§∂‡•Ä ‡§Æ‡§æ‡§Ç ‡§ï‡§æ ‡§¨‡•á‡§ü‡§æ ‡§ï‡§≠‡•Ä ‡§¶‡•á‡§∂‡§≠‡§ï‡•ç‡§§ ‡§®'''))
print(predict('''she is a bloody bitch'''))

0
1
1


In [ ]:
sentnc = '''‡§≠‡§æ‡§∞‡§§ ‡§®‡•á ‡§ï‡§π‡§æ, ‡§ö‡•Ä‡§® ‡§ï‡•á ‡§∏‡•à‡§®‡§ø‡§ï‡•ã‡§Ç ‡§ï‡•á ‡§∏‡§æ‡§• ‡§≤‡§¶‡•ç‡§¶‡§æ‡§ñ ‡§Æ‡•á‡§Ç ‡§ù‡§°‡§º‡§™, ‡§ï‡•ç‡§Ø‡§æ ‡§ï‡§π‡§®‡§æ ‡§π‡•à ‡§ö‡•Ä‡§® ‡§ï‡§æ?
‡§∏‡•ç‡§ü‡•ã‡§∞‡•Ä: ‡§ü‡•Ä‡§Æ ‡§¨‡•Ä‡§¨‡•Ä‡§∏‡•Ä
‡§Ü‡§µ‡§æ‡§ú‡§º: ‡§≠‡§∞‡§§ ‡§∂‡§∞‡•ç‡§Æ‡§æ https://t.co/Re6GyZVmbY'''
sentnc = filtertext(sentnc)
print(sentnc)
sentnc = translate(sentnc)
print(sentnc)
sentnc = preprocess_sentence(sentnc)
print(sentnc)

‡§≠‡§æ‡§∞‡§§ ‡§®‡•á ‡§ï‡§π‡§æ ‡§ö‡•Ä‡§® ‡§ï‡•á ‡§∏‡•à‡§®‡§ø‡§ï‡•ã‡§Ç ‡§ï‡•á ‡§∏‡§æ‡§• ‡§≤‡§¶‡•ç‡§¶‡§æ‡§ñ ‡§Æ‡•á‡§Ç ‡§ù‡§°‡§º‡§™ ‡§ï‡•ç‡§Ø‡§æ ‡§ï‡§π‡§®‡§æ ‡§π‡•à ‡§ö‡•Ä‡§® ‡§ï‡§æ ‡§∏‡•ç‡§ü‡•ã‡§∞‡•Ä ‡§ü‡•Ä‡§Æ ‡§¨‡•Ä‡§¨‡•Ä‡§∏‡•Ä ‡§Ü‡§µ‡§æ‡§ú‡§º ‡§≠‡§∞‡§§ ‡§∂‡§∞‡•ç‡§Æ‡§æ
india ‡§®‡•á told said tattle china from k ‡§∏‡•à‡§®‡§ø‡§ï‡•ã‡§Ç from k society along company accompaniment companionship ‡§≤‡§¶‡•ç‡§¶‡§æ‡§ñ into during through on under at throughout by over in within with to among towards inside skirmish altercation tangle which what have it utter phrase cry declare answer bid run by talk provide mention volunteer say spell out advertise drop speak up for call tell breathe speak to crack rap out take up with run past speak 
to address raise speak call on observe communicate have china in am  iana of ‡§∏‡•ç‡§ü‡•ã‡§∞‡•Ä team ‡§¨‡•Ä‡§¨‡•Ä‡§∏‡•Ä voice song alauda arvensis skylark alauda 
arvensis ‡§∂‡§∞‡•ç‡§Æ‡§æ
india ‡§®‡•á told said tattl china k ‡§∏‡•à‡§®‡§ø‡§ï‡•ã‡§Ç k societi along compa

In [ ]:
from textblob import TextBlob

def detect(query):
    """
    Detects offensive queries in Hindi using TextBlob.

    Args:
        query (str): The input query in Hindi.

    Returns:
        bool: True if the query is offensive, False otherwise.
    """
    # Create a TextBlob object for the input query
    blob = TextBlob(query)

    # Check for offensive language using TextBlob's sentiment polarity
    polarity = blob.sentiment.polarity

    # If polarity is less than 0, the query is considered offensive
    if polarity < 0:
        return 1
    else:
        return 0

correct = 0
tp = 0
fp = 0
tn = 0
fn = 0
for sen, off in zip(X_test, y_test):
    pred = max(predict(sen), detect(sen))
    if pred == 1:
      if(off == 1):
        tp += 1
        correct += 1
      else:
        fp += 1
    else:
      if(off == 0):
        correct += 1
        tn += 1
      else:
        fn += 1
accuracy = correct / len(y_test)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Accuracy:", accuracy)
print("Percision:", precision)
print("Recall:", recall)
f1_score = (2 * precision * recall) / (precision + recall)
print("F1 score:", f1_score)